# Descrição dos dados

<p align='justify'> As cardiopatias congênitas têm como causa diversos fatores, dentre eles as condições maternas durante a gestação. A identificação, o diagnóstico e o
tratamento das cardiopatias são decorrentes das ações de saúde desenvolvidas por uma equipe multidisciplinar, onde cada profissional tem responsabilidade quanto ao desfecho produzido. A maioria das cardiopatias congênitas pode ser evitada e suas repercussões minimizadas com assistência pré-concepcional, pré-natal e planejamento familiar adequado. Sendo assim, buscou-se neste estudo a identificação dos fatores de risco maternos associados à ocorrência deste evento por meio da resposta ao seguinte questionamento: Há associação entre os fatores de riscos maternos e as cardiopatias congênitas? O estudo analisou 22 casos de recém-nascidos diagnosticados com cardiopatia congênita no HUUFMA, nos anos de 2019 e 2020. </p>

*   Óbito: indica se o recém-nascido veio a óbito ou não
*   Idade: indica a idade da mãe
*   Uso de medicamentos: indica se a mãe faz uso de medicamentos ou não
*   Aborto: indica se a mãe já teve aborto ou não
*   Comorbidades: indica se a mãe tem comorbidade ou não
*   Ocupação: indica o atual cenário da mãe
*   Local de residência: indica o atual local de residência da mãe 
*   Números de consultas de pré-natal: indica a quantidade de consultas de pré-natal
*   Sexo: indica o sexo do bebê
*   Raça: indica a raça do bebê
*   Idade gestacional: indica o tempo de gestação da mãe
*   APGAR 1 minuto: indica o valor medido no primeiro minuto do bebê
*   Peso ao nascer: indica o peso de nascimento do bebê












# Leitura dos dados



In [381]:
import pandas as pd

df = pd.read_excel("dados.xlsx")

df.head()

,Obito,Idade,Uso de medicamentos,Abortos,Comorbidades,Ocupação,Local de residência,Número de consultas de pré-natal,Sexo,Raça,Idade gestacional,Apgar 1 min,Peso ao nascer
0,0,24,não,A0,não,trabalho remunerado,São Luis,4,Feminino,parda,37s e 3d,9,"2,765g"
1,0,26,não,A0,não,trabalho remunerado,Raposa,7,Feminino,Não informado,38s e 3d,7,"3,190g"
2,0,36,não,A0,não,Do lar,São José de Ribamar,10,Masculino,Não informado,38s e 4d,7,"3,110g"
3,0,31,"sim, 1 medicamento",A0,"sim, 1 comorbidade",Não informado,São Luis,10,Masculino,pardo,38s e 5d,9,"3,116g"
4,0,42,"sim, 3 medicamentos",A0,"sim, 3 comorbidades",do lar,São Luis,7,Feminino,parda,37s e 3d,5,"4,425g"


# Funções complementares

In [382]:
def get_lower(df, cols):  
    for col in cols:  
        df[col] = df[col].str.lower()

In [383]:
def remove_accent(df, cols):
    for col in cols:
      df[col] = df[col].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [384]:
def remove_parentheses(df, cols):
    for col in cols:
        df[col] = df[col].str.replace(r'\(.*\)', '')

In [385]:
def check_substring(df, cols, string):  
    for col in cols:  
       df[col] = df[col].str.contains(string, case=False)        

In [386]:
def convert_to_int(df, cols):  
    for col in cols: 
        df[col] = df[col].astype(int)

In [387]:
def convert_to_float(df, cols):  
    for col in cols: 
        df[col] = df[col].astype(float)

In [388]:
def replace_char(df, cols, char_old, char_actual):
    for col in cols:
        df[col] = df[col].str.replace(char_old, char_actual)

In [389]:
def remove_blank_spaces(df, cols):  
    for col in cols: 
        df[col] = df[col].str.strip()

In [390]:
def remove_specific_char(df, cols, char):
    for col in cols:
        df[col] = df[col].str.replace(char, '')

In [391]:
def delete_last_char(df, cols):
    for col in cols:
        df[col] = df[col].str[:-1]

In [392]:
def split_string(df, cols, char):  
    for col in cols:  
        df[col] = df[col].str.split(char, expand=True)[0]

In [393]:
def get_intervals(df, cols, n = 3):
    for col in cols:
        quantile = pd.qcut(df[col], n, labels=False, retbins=False)    
        for i in range(n):
            minn = df[col].loc[quantile == i].min()
            maxx = df[col].loc[quantile == i].max()
            qnt = df[col].loc[quantile == i].count()
            
            print(minn, maxx, qnt)

In [394]:
def analyze_dataframe(df):
    pd.set_option('display.width', 1000)

    print(df.info())

    print()
    print('Variáveis quantitativas:')
    print(df.describe().transpose())

    print()
    print('Variáveis qualitativas:')
    print(df.describe(include='O').transpose())

In [395]:
def crosstable_quality(df, cols, target, normalize=False):  
    df_list = []

    for i, col in enumerate(cols):
      df_list.append(pd.crosstab(df[col], df[target], normalize=normalize))

    return pd.concat(df_list, keys=cols)

In [396]:
def crosstable_target(df, target_variable, path = None):  
    categorical_vars = df.describe(include='O').columns

    df_desc_f = crosstable_quality(df, categorical_vars, target_variable)

    df_desc_p = crosstable_quality(df, categorical_vars, target_variable, normalize='columns').round(4)*100

    if path != None:
        with pd.ExcelWriter(path) as writer:  
            df_desc_f.to_excel(writer, sheet_name='frequencia')
            df_desc_p.to_excel(writer, sheet_name='frequencia_relativa')

# Pré-processamento dos dados

In [397]:
#Atributo Idade
df.loc[df['Idade'] <= 24, 'Idade categoria'] = 'Até 24 anos'

df.loc[(df['Idade'] >= 25) & (df['Idade'] < 35), 'Idade categoria'] = 'Entre 25 e 34 anos'

df.loc[df['Idade'] >= 35, 'Idade categoria'] = 'Maior que 35 anos'

df.drop('Idade', axis='columns', inplace = True)

df['Idade categoria'].value_counts()

Entre 25 e 34 anos    9
Maior que 35 anos     7
Até 24 anos           6
Name: Idade categoria, dtype: int64

In [398]:
#Atributo Uso de medicamentos
check_substring(df, ['Uso de medicamentos'], 'sim')

convert_to_int(df, ['Uso de medicamentos'])

mapping = {0: 'Não', 1: 'Sim'}

df['Uso de medicamentos'] = df['Uso de medicamentos'].map(mapping)

df['Uso de medicamentos'].value_counts()

Não    18
Sim     4
Name: Uso de medicamentos, dtype: int64

In [399]:
#Atributo Abortos
remove_blank_spaces(df, ['Abortos'])

remove_specific_char(df, ['Abortos'], 'A')

replace_char(df, ['Abortos'], '2', '1')

convert_to_int(df, ['Abortos'])

mapping = {0: 'Não', 1: 'Sim'}

df['Abortos'] = df['Abortos'].map(mapping)

df['Abortos'].value_counts()

Não    18
Sim     4
Name: Abortos, dtype: int64

In [400]:
#Atributo Comorbidades
check_substring(df, ['Comorbidades'], 'sim')

convert_to_int(df, ['Comorbidades'])

mapping = {0: 'Não', 1: 'Sim'}

df['Comorbidades'] = df['Comorbidades'].map(mapping)

df['Comorbidades'].value_counts()

Não    16
Sim     6
Name: Comorbidades, dtype: int64

In [401]:
#Atributo Ocupação
get_lower(df, ['Ocupação'])

df['Ocupação'].value_counts()

trabalho remunerado    10
do lar                  5
estudante               5
não informado           2
Name: Ocupação, dtype: int64

In [402]:
#Atributo Local de residência
get_lower(df, ['Local de residência'])

df['Local de residência'].value_counts()

são luis               17
são josé de ribamar     4
raposa                  1
Name: Local de residência, dtype: int64

In [403]:
#Atributo Número de consultas de pré-natal
df.loc[df['Número de consultas de pré-natal'] <= 7, 'Prenatal'] = 'Até 7 consultas'

df.loc[(df['Número de consultas de pré-natal'] >= 8) & (df['Número de consultas de pré-natal'] < 10), 'Prenatal'] = '8 e 9 consultas'

df.loc[df['Número de consultas de pré-natal'] >= 10, 'Prenatal'] = 'Maior que 10 consultas'

df.drop('Número de consultas de pré-natal', axis='columns', inplace = True)

df['Prenatal'].value_counts()

Até 7 consultas           10
Maior que 10 consultas     6
8 e 9 consultas            6
Name: Prenatal, dtype: int64

In [404]:
#Atributo Sexo
get_lower(df, ['Sexo'])

remove_specific_char(df, ['Sexo'], '\t')

remove_blank_spaces(df, ['Sexo'])

df['Sexo'].value_counts()

feminino     11
masculino    11
Name: Sexo, dtype: int64

In [405]:
#Atributo Raça
get_lower(df, ['Raça'])

delete_last_char(df, ['Raça'])

df['Raça'].value_counts()

não informad    11
pard             8
branc            3
Name: Raça, dtype: int64

In [406]:
#Atributo Idade gestacional
split_string(df, ['Idade gestacional'], 's')

convert_to_int(df, ['Idade gestacional'])

df.loc[df['Idade gestacional'] < 37, 'Semanas'] = 'Até 37 semanas'

df.loc[df['Idade gestacional'] >= 37, 'Semanas'] = 'Maior que 37 semanas'

df.drop('Idade gestacional', axis='columns', inplace = True)

df['Semanas'].value_counts()

Maior que 37 semanas    16
Até 37 semanas           6
Name: Semanas, dtype: int64

In [407]:
#Atributo APGAR
df.loc[df['Apgar 1 min'] < 7, 'Apgar 1 min categoria'] = 'Abaixo de 7'

df.loc[df['Apgar 1 min'] >= 7, 'Apgar 1 min categoria'] = 'Igual ou maior que 7'

df.drop('Apgar 1 min', axis='columns', inplace = True)

df['Apgar 1 min categoria'].value_counts()

Igual ou maior que 7    15
Abaixo de 7              7
Name: Apgar 1 min categoria, dtype: int64

In [408]:
#Atributo Peso ao nascer
remove_specific_char(df, ['Peso ao nascer'], 'g')

remove_blank_spaces(df, ['Peso ao nascer'])

replace_char(df, ['Peso ao nascer'], ',', '.')

convert_to_float(df, ['Peso ao nascer'])

df.loc[(df['Peso ao nascer'] >= 1.0) & (df['Peso ao nascer'] < 1.5), 'Peso categoria'] = 'Muito abaixo do peso'

df.loc[(df['Peso ao nascer'] >= 1.5) & (df['Peso ao nascer'] < 2.5), 'Peso categoria'] = 'Abaixo do peso'

df.loc[(df['Peso ao nascer'] >= 2.5) & (df['Peso ao nascer'] < 4), 'Peso categoria'] = 'Peso adequado'

df.loc[df['Peso ao nascer'] >= 4, 'Peso categoria'] = 'Elevado'

df.drop('Peso ao nascer', axis='columns', inplace = True)

df['Peso categoria'].value_counts()

Peso adequado           15
Abaixo do peso           5
Elevado                  1
Muito abaixo do peso     1
Name: Peso categoria, dtype: int64

# Análise dos dados

In [409]:
analyze_dataframe(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Obito                  22 non-null     int64 
 1   Uso de medicamentos    22 non-null     object
 2   Abortos                22 non-null     object
 3   Comorbidades           22 non-null     object
 4   Ocupação               22 non-null     object
 5   Local de residência    22 non-null     object
 6   Sexo                   22 non-null     object
 7   Raça                   22 non-null     object
 8   Idade categoria        22 non-null     object
 9   Prenatal               22 non-null     object
 10  Semanas                22 non-null     object
 11  Apgar 1 min categoria  22 non-null     object
 12  Peso categoria         22 non-null     object
dtypes: int64(1), object(12)
memory usage: 2.4+ KB
None

Variáveis quantitativas:
       count  mean       std  min  25%  50%  75%  

In [410]:
crosstable_target(df, 'Obito', 'descriptive_analysis.xlsx')

FREQUENCIA:_____________________________________________________________________________________________
Obito                                          0   1
Uso de medicamentos   Não                      8  10
                      Sim                      3   1
Abortos               Não                     10   8
                      Sim                      1   3
Comorbidades          Não                      8   8
                      Sim                      3   3
Ocupação              do lar                   2   3
                      estudante                3   2
                      não informado            2   0
                      trabalho remunerado      4   6
Local de residência   raposa                   1   0
                      são josé de ribamar      2   2
                      são luis                 8   9
Sexo                  feminino                 5   6
                      masculino                6   5
Raça                  branc                    